In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

# =====================================================
# 1. LOAD DATA
# =====================================================

sales_df = pd.read_csv("/kaggle/input/cleaned-1/prod_store_sales_header (1).csv")
customers_df = pd.read_csv("/kaggle/input/remaining-data/customer_details (1).csv")
loyalty_rules_df = pd.read_csv("/kaggle/input/remaining-data/loyalty_rules (1).csv")

# Ensure numeric
sales_df["total_amount"] = sales_df["total_amount"].astype(float)

# =====================================================
# 2. SCHEMA STANDARDIZATION (PROFESSIONAL STEP)
# =====================================================
# Preserve original column, add semantic alias

if "points_per_unit_spend" not in loyalty_rules_df.columns:
    raise ValueError("Expected column 'points_per_unit_spend' missing in loyalty_rules")

loyalty_rules_df["points_per_rupee"] = loyalty_rules_df["points_per_unit_spend"]

# Optional bonus columns (safe defaults)
if "bonus_threshold" not in loyalty_rules_df.columns:
    loyalty_rules_df["bonus_threshold"] = None

if "bonus_points" not in loyalty_rules_df.columns:
    loyalty_rules_df["bonus_points"] = 0

# =====================================================
# 3. READ LOYALTY RULES
# =====================================================

rule = loyalty_rules_df.iloc[0]

POINTS_PER_RUPEE = rule["points_per_rupee"]
BONUS_THRESHOLD = rule["bonus_threshold"]
BONUS_POINTS = rule["bonus_points"]

# =====================================================
# 4. CALCULATE LOYALTY POINTS PER TRANSACTION
# =====================================================

def calculate_points(amount):
    points = amount * POINTS_PER_RUPEE

    if pd.notna(BONUS_THRESHOLD) and amount >= BONUS_THRESHOLD:
        points += BONUS_POINTS

    return int(points)

sales_df["earned_points"] = sales_df["total_amount"].apply(calculate_points)

# =====================================================
# 5. TRANSACTION-LEVEL OUTPUT
# =====================================================

txn_points_df = sales_df[
    [
        "transaction_id",
        "customer_id",
        "transaction_date",
        "total_amount",
        "earned_points"
    ]
]

# =====================================================
# 6. AGGREGATE POINTS PER CUSTOMER
# =====================================================

customer_points = (
    txn_points_df
    .groupby("customer_id")["earned_points"]
    .sum()
    .reset_index()
    .rename(columns={"earned_points": "new_points"})
)

# =====================================================
# 7. UPDATE CUSTOMER MASTER
# =====================================================

if "loyalty_points" not in customers_df.columns:
    customers_df["loyalty_points"] = 0

customers_df = customers_df.merge(
    customer_points,
    on="customer_id",
    how="left"
)

customers_df["new_points"] = customers_df["new_points"].fillna(0)
customers_df["loyalty_points"] += customers_df["new_points"]

customers_df.drop(columns=["new_points"], inplace=True)

# =====================================================
# 8. SAVE OUTPUTS
# =====================================================

txn_points_df.to_csv("transaction_loyalty_points.csv", index=False)
customers_df.to_csv("customer_details_updated.csv", index=False)

print("✅ Loyalty Point Calculation Engine executed successfully.")


✅ Loyalty Point Calculation Engine executed successfully.
